In [1]:
# 1. STOCK ANALYSIS AND PORTFOLIO MANAGEMENT REPORT 

# This report summarizes the findings from the analysis of stock price data for a selected universe of technology stocks, including IBM, MSFT, GOOG, 
# AAPL, AMZN, META, NFLX, TSLA, ORCL, and SAP. The analysis focuses on calculating dividends, daily price changes, portfolio management strategies, and 
# performance metrics. 

# The objective is to manage a $5 million fund using two distinct rebalancing strategies—"buying low" and "buying high"—and to assess the performance 
# of each strategy over the course of 2018.


# Load necessary libraries
library(dplyr)
 install.packages("lubridate")
  library(lubridate)
 library(data.table)
 library(tidyr)
 options(repr.matrix.max.rows=600, repr.matrix.max.cols=200)
  

Warning message:
"package 'dplyr' was built under R version 4.4.2"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Installing package into 'C:/Users/anast/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



package 'lubridate' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'lubridate'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\anast\AppData\Local\R\win-library\4.4\00LOCK\lubridate\libs\x64\lubridate.dll to C:\Users\anast\AppData\Local\R\win-library\4.4\lubridate\libs\x64\lubridate.dll: Permission denied"
Warning message:
"restored 'lubridate'"



The downloaded binary packages are in
	C:\Users\anast\AppData\Local\Temp\RtmpYlM4gJ\downloaded_packages



Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union



Attaching package: 'data.table'


The following objects are masked from 'package:lubridate':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from 'package:dplyr':

    between, first, last




In [2]:
# 2. Data Preparation. Download and read data. 


# Historical stock data for the selected companies was sourced from Yahoo Finance. Each stock's historical prices were downloaded in CSV format, with a # focus on the "Close" and "Adj Close" prices for analysis. The following stocks were included in the analysis: IBM, MSFT, GOOG, AAPL, AMZN,
# META, NFLX, ORCL, SAP, TSLA. 

IBM = read.csv("IBM.csv")
 MSFT = read.csv("MSFT.csv")
 GOOG = read.csv("GOOG.csv")
 AAPL = read.csv("AAPL.csv")
 AMZN = read.csv("AMZN.csv")
 META = read.csv("META.csv")
 NFLX = read.csv("NFLX.csv")
 ORCL = read.csv("ORCL.csv")
 SAP = read.csv("SAP.csv")
 TSLA = read.csv("TSLA.csv")

# Check the first few rows of the Date column
# Convert Date column before calling cal_div
IBM$Date <- as.Date(IBM$Date, format = "%d-%b-%y")
MSFT$Date <- as.Date(MSFT$Date, format = "%d-%b-%y")
GOOG$Date <- as.Date(GOOG$Date, format = "%d-%b-%y")
AAPL$Date <- as.Date(AAPL$Date, format = "%d-%b-%y")
AMZN$Date <- as.Date(AMZN$Date, format = "%d-%b-%y")
META$Date <- as.Date(META$Date, format = "%d-%b-%y")
NFLX$Date <- as.Date(NFLX$Date, format = "%d-%b-%y")
ORCL$Date <- as.Date(ORCL$Date, format = "%d-%b-%y")
SAP$Date <- as.Date(SAP$Date, format = "%d-%b-%y")
TSLA$Date <- as.Date(TSLA$Date, format = "%d-%b-%y")

str(IBM)


Warning message in file(file, "rt"):
"cannot open file 'IBM.csv': No such file or directory"


ERROR: Error in file(file, "rt"): cannot open the connection


In [ ]:
#3. Corporations generally issue stock dividends on some days. The total dividend you get
#on such a day is the stock dividend times your shares if you have shares of this stock on
#the dividend day. If you buy shares on the dividend day, these bought shares are not
#qualified to get dividend. If you sell shares on the dividend day, the sold shares are
#qualified to get dividend. For example, on 2/8/2018, IBM issued 1.5 dividend per share.
#Calculating the dividend.

cal_div = function(df){
    df_shift = setDF(shift(df, n=1L, fill=NA, type=c("lag"), give.names=TRUE))
    df_new = data.frame(df["Date"], df["Close"],
 round(df_shift["Close_lag_1"]/df["Close"], 6),
 round(df_shift["Adj.Close_lag_1"]/df["Adj.Close"], 6))
    df_new[is.na(df_new)] = 0
    df_Div = df_new[(df_new$Close_lag_1 - df_new$Adj.Close_lag_1) > 0.00001,]
    df_Div = data.frame(df_new["Date"], df_new["Close"], Div = (df_new$Close_lag_1 -
df_new$Adj.Close_lag_1 >0.00001) * (df_new$Close_lag_1 - df_new$Adj.Close_lag_1))
    df_Div = transmute(df_Div, Date = Date, Dividend = round(Close * Div, 2))
    return (df_Div)
 }




In [ ]:
#Days Shift
days = 5

In [ ]:
#Calculate price changes according to the days shift
cal_change = function(days, df){
    before = df[1, "Adj.Close"]
    df_new = data.frame()
    for(i in seq(1,nrow(df),days)){
        after = df[i, "Adj.Close"]
        df_new[i,1] = after/before
        before = after
    }
    df_new[is.na(df_new)] = 0
    if(nrow(df_new) != nrow(df)){
        for(i in (nrow(df_new)+1):250){
            df_new[i,1] = 0
        }
    }
    return (df_new)
 }

In [ ]:
# Creating the Universe: 
# The final dataset comprised 10 CSV files, each containing daily stock price information for 2018. A comprehensive DataFrame was created, 
# consolidating the "Close" and "Adj Close" prices into 20 columns, with each row representing daily prices, and dividends. 


universe_change = function(universe, days) {
    universe[,5] = cal_change(days, IBM)
    universe[,9] = cal_change(days, MSFT)
    universe[,13] = cal_change(days, GOOG)
    universe[,17] = cal_change(days, AAPL)
    universe[,21] = cal_change(days, AMZN)
    universe[,25] = cal_change(days, META)
    universe[,29] = cal_change(days, NFLX)
    universe[,33] = cal_change(days, ORCL)
    universe[,37] = cal_change(days, SAP)
    universe[,41] = cal_change(days, TSLA)
    return (universe)
 }

In [ ]:
 #IBM_change = cal_change(days, IBM)
 #MSFT_change = cal_change(days, MSFT)
 #GOOG_change = cal_change(days, GOOG)
 #AAPL_change = cal_change(days, AAPL)
 #AMZN_change = cal_change(days, AMZN)
 #META_change = cal_change(days, META)
 #NFLX_change = cal_change(days, NFLX)
 #ORCL_change = cal_change(days, ORCL)
 #SAP_change = cal_change(days, SAP)
 #TSLA_change = cal_change(days, TSLA)

In [ ]:
#Clear data for USD_JPN
USD_JPY = read.csv("USDJPY_historical_data.csv")[,c(1,5)]
USD_JPY$Date = as.Date(USD_JPY$Date, format="%d-%b-%y")
USD_JPY = rename(USD_JPY, JPY_Close = Closeÿ)
str(USD_JPY)



In [ ]:
#Creating Detail for Universe
Create_Universe = function() {
  # Calculating dividends for each stock
  IBM_Div = cal_div(IBM)
  MSFT_Div = cal_div(MSFT)
  GOOG_Div = cal_div(GOOG)
  AAPL_Div = cal_div(AAPL)
  AMZN_Div = cal_div(AMZN)
  META_Div = cal_div(META)
  NFLX_Div = cal_div(NFLX)
  ORCL_Div = cal_div(ORCL)
  SAP_Div = cal_div(SAP)
  TSLA_Div = cal_div(TSLA)

  # Calculating daily percentage changes for each stock
  IBM_change = cal_change(days, IBM)
  MSFT_change = cal_change(days, MSFT)
  GOOG_change = cal_change(days, GOOG)
  AAPL_change = cal_change(days, AAPL)
  AMZN_change = cal_change(days, AMZN)
  META_change = cal_change(days, META)
  NFLX_change = cal_change(days, NFLX)
  ORCL_change = cal_change(days, ORCL)
  SAP_change = cal_change(days, SAP)
  TSLA_change = cal_change(days, TSLA)

  universe = data.frame(IBM["Date"])

universe = mutate(universe, IBM_Close = IBM$Close, IBM_Adj = IBM$Adj.Close,
IBM_Dividend = IBM_Div$Dividend, IBM_ch = IBM_change[,1])
universe = mutate(universe, MSFT_Close = MSFT$Close, MSFT_Adj = MSFT$Adj.Close,
MSFT_Dividend = MSFT_Div$Dividend, MSFT_ch = MSFT_change[,1])
universe = mutate(universe, GOOG_Close = GOOG$Close, GOOG_Adj = GOOG$Adj.Close,
GOOG_Dividend = GOOG_Div$Dividend, GOOG_ch = GOOG_change[,1])

universe = mutate(universe, AAPL_Close = AAPL$Close, AAPL_Adj = AAPL$Adj.Close,
AAPL_Dividend = AAPL_Div$Dividend, AAPL_ch = AAPL_change[,1])
universe = mutate(universe, AMZN_Close = AMZN$Close, AMZN_Adj = AMZN$Adj.Close,
AMZN_Dividend = AMZN_Div$Dividend, AMZN_ch = AMZN_change[,1])
universe = mutate(universe, META_Close = META$Close, META_Adj = META$Adj.Close,
META_Dividend = META_Div$Dividend, FB_ch = FB_change[,1])
universe = mutate(universe, NFLX_Close = NFLX$Close, NFLX_Adj = NFLX$Adj.Close,
NFLX_Dividend = NFLX_Div$Dividend, NFLX_ch = NFLX_change[,1])
universe = mutate(universe, ORCL_Close = ORCL$Close, ORCL_Adj = ORCL$Adj.Close,
ORCL_Dividend = ORCL_Div$Dividend, ORCL_ch = ORCL_change[,1])
universe = mutate(universe, SAP_Close = SAP$Close, SAP_Adj = SAP$Adj.Close,
SAP_Dividend = SAP_Div$Dividend, SAP_ch = SAP_change[,1])
universe = mutate(universe, TSLA_Close = TSLA$Close, TSLA_Adj = TSLA$Adj.Close,
TSLA_Dividend = TSLA_Div$Dividend, TSLA_ch = TSLA_change[,1])
 
  # Converting Date format and merging with USD/JPY data
  universe$Date = as.Date(universe$Date, format="%d-%m-%Y")
  universe = merge(universe, USD_JPY, by="Date")
  
  return(universe)
}


In [ ]:
universe = Create_Universe() 
universe



In [ ]:
#4. 5 business days later on Jan 17 (Jan 15 was a holiday), you re-check the market and adjust your portfolio again. You will have a new portfolio on Jan 17.
#The first day (01/02/2018)
#['IBM','MSFT', 'GOOG&', 'AAPL', 'AMZN']

holdstock = c(1:5) 
buystock = c(1:5)
#portfolio : cash, stock1_shares, stock2_shares,..., stock5_shares
portfolio = c(5000000, 0, 0, 0, 0, 0)
row_num = 1;


In [ ]:
buy_stock = function(buystock, p)
{ money_to_buy = 1:5
 money_to_buy[1] = as.integer(p[1]/5) + p[1]%%5   
  money_to_buy[2] = as.integer(p[1]/5)
 money_to_buy[3] = as.integer(p[1]/5)   
 money_to_buy[4] = as.integer(p[1]/5)
money_to_buy[5] = as.integer(p[1]/5)
  for(i in 1:5){ 
      p[i+1] = as.integer(money_to_buy[i]/universe[row_num, 4 * buystock[i] - 2])
       p[1] = p[1] - (p[i+1] * universe[row_num, 4 * buystock[i] - 2])
      }
 return(p)
 }

In [ ]:
portfolio = buy_stock(buystock, portfolio)

In [ ]:
portfolio

In [ ]:
holdstock = buystock

In [ ]:
#5. Daily Mark-to-Market (MTM) Calculation: 
# The calculate_mtm function was developed to track the MTM value of the portfolio, factoring in cash holdings and stock prices over time.   
# MTM values were calculated daily, reflecting changes in stock prices and any dividends received. The MTM series for both strategies was tracked 
# throughout the year.


#calculate MTM
cal_MTM = function(p){
    MTM = 0
     for(i in 1:5){
         MTM = MTM + p[i+1] * universe[row_num, (holdstock[i]*4-2)]
         }
    MTM = MTM + p[1]   
    return (MTM)
    }


In [ ]:
MTM_Series = 5000000
MTM_Series

In [ ]:
#Sale of all the stocks
sale_stock = function(holdstock, p) {
     for(i in 1:5){
          p[1] = p[1] + p[i+1]*universe[row_num, holdstock[i]*4-2]
          p[i+1] = 0
         }
    return(p)
    }


In [ ]:
#Check dividend
check_div = function(holdstock, p) {
     earn_dividend = 0
     for(i in 1:5){
          if(universe[row_num, holdstock[i]*4] != 0){
              earn_dividend = earn_dividend + p[i+1] * universe[row_num, holdstock[i]*4]
              }
         }
    return(earn_dividend)
    }


In [ ]:
#find where the drop is the most.
find_drop = function(){
    change = c()
    for(i in 1:10){
        change = c(change, universe[row_num, i*4+1])
        }
    return(order(change))
    }

In [ ]:
#6. If you run this strategy every 5 days all the way to Dec 31 2018, you will have a daily MTM
#You expect the MTM on Dec 31 2018 should be higher than 5m because you
#always buy the stocks that dropped the most, i.e., you always buy low.
#Calculate 5 days rebalancing for buy low

for(i in 2:nrow(universe)){
    print(row_num)
    print(universe[row_num,1])
    earn_div = 0
    earn_div = check_div(holdstock, portfolio)
    portfolio[1] = portfolio[1] + earn_div
    if(row_num %% days == 1){ ## need to change portfolio
         print("")
         portfolio = sale_stock(holdstock, portfolio)
        buystock = c(find_drop()[1:5])
         portfolio = buy_stock(buystock, portfolio)
          holdstock = buystock
        }
     MTM_Series = c(MTM_Series, cal_MTM(portfolio))
     print(holdstock)
     print(portfolio)
    }

In [ ]:
MTM_Series
length(MTM_Series)

In [ ]:
#7. You will create a "high tech index" which is simply the daily average of the 10 stocks "Close" prices.
#Compare your MTM series with the "high tech index" and plot their curves. To plot the
#two curves together, you may want to convert the series to daily percentage change with
#regard to Jan 02 2018
#Calculate high tech index
close_price =
cbind(universe[,2],universe[,6],universe[,10],universe[,14],universe[,18],
      universe[,22],universe[,26],universe[,30],universe[,34],universe[,38])
close_price

In [ ]:
high_tech_index = apply(close_price, 1, mean)
#high_tech_index


In [ ]:
#Plot curve for buying high and high tech index
daily_change_mtm = 0
for (i in 1:length(MTM_Series)){
     daily_change_mtm[i] = MTM_Series[i]/MTM_Series[1]
    }
daily_change_index = 0
for (i in 1:length(high_tech_index)){
     daily_change_index[i] = high_tech_index[i]/high_tech_index[1]
    }



In [ ]:
day = 1:length(universe[,1])
options(repr.plot.width = 10, repr.plot.height = 10)
plot(day,daily_change_mtm,type='line',col="blue", pch="o", lty=1, ylim=c(0.9,1.4), lwd=2)
lines(day, daily_change_index, col="red",lty=1, lwd = 2)
box(lwd=4)

In [ ]:
#8. Download the USD/JPY 2018 historical data at https://www.myfxbook.com/en/forex-market/currencies/USDJPY-historical-data
#then use the "Close" column as the rate to convert your MTM series from USD to JPY.
#Plot the two MTM curves. You will need to convert to daily percentage change too. 
#Translate USD to JPY and Plot the JPY MTM for Buying low

JPY_MTM_Series = MTM_Series * universe[,'JPY_Close']


In [ ]:
daily_change_JPY_mtm = 0
for (i in 1:length(JPY_MTM_Series)){
     daily_change_JPY_mtm[i] = JPY_MTM_Series[i]/JPY_MTM_Series[1]
    }
daily_change_JPY_mtm

In [ ]:
day = 1:length(universe[,1])

plot(day,daily_change_mtm,type='line',col="blue", pch="o", lty=1, ylim=c(0.8,1.3), lwd = 2)

lines(day, daily_change_JPY_mtm, col="red",lty=1, lwd = 2)

box(lwd=4)

In [ ]:
#9. The above two strategies both rebalance every 5 days. Try to change the days interval and find the optimal days interval that maximizes the MTM on 12/31/2018.
# Portfolio Management Strategy. Initial Portfolio Setup. 

# An initial investment of $5,000,000 was allocated  allocated equally among selected stocks (IBM, MSFT, GOOG, AAPL, and AMZN). Each stock received an # equal share of the investment.
# Each stock received $1 million. Based on the closing prices, the number of shares purchased and the remaining cash in the zero-interest account were # calculated.

#Calculate the optimal days
for(d in 1:nrow(universe)){
days = d
## initialize
universe = universe_change(universe, d)
 holdstock = c(1:5)
buystock = c(1:5)
portfolio = c(5000000, 0, 0, 0, 0, 0)
MTM_Series = NA
    for(i in 1:nrow(universe)){
        row_num = i
        #print(universe[row_num,1])


#10. Dividend Earnings: 
# Dividends were calculated for each stock based on the price data, contributing to the overall returns of the portfolio.        
earn_div = 0
earn_div = check_div(holdstock, portfolio)
portfolio[1] = portfolio[1] + earn_div
        if(days == 1) {
            portfolio = sale_stock(holdstock, portfolio)
            buystock = c(find_drop()[1:5])
            portfolio = buy_stock(buystock, portfolio)
            holdstock = buystock
         }else if(row_num %% days == 1){ ## need to change portfolio
            #print("")
            portfolio = sale_stock(holdstock, portfolio)
            buystock = c(find_drop()[1:5])
            portfolio = buy_stock(buystock, portfolio)
            holdstock = buystock
            }
            MTM_Series = c(MTM_Series, cal_MTM(portfolio))
        #print(holdstock)
        #print(portfolio)
}
    MTM_Series = na.omit(MTM_Series)
    if(d == 1){
        Last_MTM_Series = MTM_Series[length(MTM_Series)]
        }else {
        Last_MTM_Series = c(Last_MTM_Series, MTM_Series[length(MTM_Series)])
            }
    }
        length(Last_MTM_Series)

In [ ]:
Last_MTM_Series
order(Last_MTM_Series)[length(Last_MTM_Series)]

In [ ]:
#optimal day of interval is 193

In [ ]:
#11. Another strategy is "5 days rebalancing of buying high" You always buy the 5 stocks
#whose "Adj Close" prices surge the most in terms of percentage because you believe the
#trend will continue. Run the new strategy and see how the MTM will change.
#Calculate 5 days rebalancing for buy high

days = 5

universe = universe_change(universe, days)
holdstock = c(1:5)
buystock = c(1:5)

portfolio = c(5000000, 0, 0, 0, 0, 0)

MTM_Series_High = NA
for(i in 1:nrow(universe)){
row_num = i
earn_div = 0

earn_div = check_div(holdstock, portfolio)
portfolio[1] = portfolio[1] + earn_div
    if(row_num %% days == 1){ ## need to change portfolio



portfolio = sale_stock(holdstock, portfolio)
        buystock = c(find_drop()[10:6])
        #print(buystock)
        portfolio = buy_stock(buystock, portfolio)
        holdstock = buystock
}
    MTM_Series_High = c(MTM_Series_High, cal_MTM(portfolio))

}

MTM_Series_High = na.omit(MTM_Series_High)

In [ ]:
MTM_Series_High

In [ ]:
#12. Portfolio Performance:
# The performance of the portfolio was monitored using the MTM series. 
# A "High Tech Index" was computed as the daily average of the "Close" prices of all ten stocks, providing a benchmark for performance comparison.

#Plot curve for buying high and high tech index
daily_change_mtm_high = 0
for (i in 1:length(MTM_Series_High)){
    daily_change_mtm_high[i] = MTM_Series_High[i]/MTM_Series_High[1]

}
daily_change_index_high = 0
for (i in 1:length(high_tech_index)){

daily_change_index_high[i] = high_tech_index[i]/high_tech_index[1]
    }

In [ ]:
day = 1:length(universe[,1])

options(repr.plot.width = 10, repr.plot.height = 10)
plot(day,daily_change_mtm_high,type= 'line',col="blue", pch="o", lty=1, ylim=c(0.9,1.4),lwd = 2)
lines(day, daily_change_index_high, col="red",lty=1, lwd = 2)

box(lwd=4)

In [ ]:
#Translate USD to JPY and Plot the JPY MTM for Buying high
JPY_MTM_Series_High = MTM_Series_High * universe[,'JPY_Close']

In [ ]:
daily_change_JPY_mtm_high = 0
for (i in 1:length(JPY_MTM_Series)){
    daily_change_JPY_mtm_high[i] = JPY_MTM_Series_High[i]/JPY_MTM_Series_High[1]
    }
daily_change_JPY_mtm_high

In [ ]:
day = 1:length(universe[,1])
plot(day,daily_change_mtm_high,type='line',col="blue", pch="o", lty=1, ylim=c(0.8,1.3), lwd = 2)
lines(day, daily_change_JPY_mtm_high, col="red",lty=1, lwd = 2)
box(lwd=4)

In [ ]:
#13. Calculate the optimal days interval.
#needs to be reviewed due to warning messages.
for(d in 1:nrow(universe)){
days = d
    ## initialize

universe = universe_change(universe, d)

holdstock = c(1:5)
buystock = c(1:5)

portfolio = c(5000000, 0, 0, 0, 0, 0)

MTM_Series_High = NA
    for(i in 1:nrow(universe)){
row_num = i
#print(universe[row_num,1])
earn_div = 0

earn_div = check_div(holdstock, portfolio)
portfolio[1] = portfolio[1] + earn_div
        if(days == 1) {
            portfolio = sale_stock(holdstock, portfolio)
buystock = c(find_drop()[10:6])
portfolio = buy_stock(buystock, portfolio)
holdstock = buystock
            }else if(row_num %% days == 1){ ## need to change portfolio

#print("")

portfolio = sale_stock(holdstock, portfolio)
buystock = c(find_drop()[10:6])
portfolio = buy_stock(buystock, portfolio)

holdstock = buystock
}
 MTM_Series_High = c(MTM_Series_High, cal_MTM(portfolio))

#print(holdstock)
#print(portfolio)
        }
 
MTM_Series_High = na.omit(MTM_Series_High)

if(d == 1){
    Last_MTM_Series_High = MTM_Series_High[length(MTM_Series_High)]

}else {

Last_MTM_Series_High = c(Last_MTM_Series_High,
MTM_Series_High[length(MTM_Series_High)])

}
}

In [ ]:
Last_MTM_Series_High
order(Last_MTM_Series_High)[length(Last_MTM_Series_High)]


In [ ]:
#The optimal day interval of buying high is 17.

In [ ]:
# Visualization
# Graphical representations were created to illustrate the MTM value versus the high-tech index, showcasing the portfolio's performance over time.

# Strategy Effectiveness
# Both a buy-low and a buy-high strategy were tested, yielding insights into the optimal days for rebalancing the portfolio. The performance metrics 
# indicated a potential strategy for maximizing returns through careful stock selection and timing.

# Conclusion
# The analysis effectively demonstrated how systematic portfolio management, informed by detailed stock price analysis, can lead to informed investment # decisions. The implementation of a systematic approach to rebalancing and dividend calculation allowed for an optimized investment strategy.

# The "buying low" strategy demonstrated a higher MTM by the end of the year compared to the "buying high" strategy.
# The MTM values were consistently higher than the High Tech Index, indicating effective portfolio management.
